In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, datasets


In [ ]:
!pip install pillow

In [6]:
# CHANGE USERNAME WHEN WORKING BETWEEN LAPTOP AND DESKTOP!!
import os
base_dir = "c:\\Users\\Fear2\\OneDrive - University of North Georgia\\FALL 2023\\Cyber capstone\\Experimentation\\SortedData"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
# Replace with your actual base directory path

# Replace with the actual subfolder path you want to check
test_path = os.path.join(base_dir, "train", "clubs", "ace of clubs")

print("Constructed path:", test_path)
print("Does the path exist?", os.path.isdir(test_path))

Constructed path: c:\Users\Fear2\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\clubs\ace of clubs
Does the path exist? False


In [5]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_dataset(base_dir, target_size=(224, 224)):
    images = []
    labels = []
    class_labels = {'hearts': 0, 'diamonds': 1, 'clubs': 2, 'spades': 3}
    
    # Navigate through each suit folder
    for suit in class_labels.keys():
        suit_dir = os.path.join(base_dir, suit)
        print(f"Checking suit directory: {suit_dir}")  # Debug print
        if os.path.isdir(suit_dir):
            label_index = class_labels[suit]
            print(f"Label index for {suit}: {label_index}")  # Debug print

            # Navigate through each card folder within the suit folder
            for card_folder in os.listdir(suit_dir):
                card_folder_path = os.path.join(suit_dir, card_folder)
                #print(f"Checking card folder: {card_folder_path}")  # Add this debug print
                if os.path.isdir(card_folder_path):
                    # Load each image from the card folder
                    for image_file in os.listdir(card_folder_path):
                        image_path = os.path.join(card_folder_path, image_file)
                        image = load_img(image_path, target_size=target_size)
                        #print(f"Loading image: {image_path}")  # Debug print
                        image_array = img_to_array(image)
                        images.append(image_array)
                        labels.append(label_index)
    
    # After loading all images and labels, shuffle them
    combined = list(zip(images, labels))
    np.random.shuffle(combined)
    images[:], labels[:] = zip(*combined)                    
    
    images = np.array(images, dtype='float32') / 255.0  # Normalize to [0, 1]
    labels = np.array(labels, dtype='int32')
    
    # Print some of the labels to verify they are correct
    print(labels[:100])
    
    return images, labels, class_labels

# Directory Definitions
base_dir = "c:\\Users\\Fear2\\OneDrive - University of North Georgia\\FALL 2023\\Cyber capstone\\Experimentation\\SortedData"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# Usage
train_images, train_labels, train_class_labels = load_dataset(train_dir)
valid_images, valid_labels, valid_class_labels = load_dataset(validation_dir)
test_images, test_labels, test_class_labels = load_dataset(test_dir)

# Check shapes and class labels
print(train_images.shape, train_labels.shape)
print(valid_images.shape, valid_labels.shape)
print(test_images.shape, test_labels.shape)
print(train_class_labels)


Checking suit directory: c:\Users\Fear2\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\hearts
Checking suit directory: c:\Users\Fear2\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\diamonds
Checking suit directory: c:\Users\Fear2\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\clubs
Checking suit directory: c:\Users\Fear2\OneDrive - University of North Georgia\FALL 2023\Cyber capstone\Experimentation\SortedData\train\spades


ValueError: not enough values to unpack (expected 2, got 0)

In [12]:
# Check the shape of the arrays
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)
print(valid_images.shape, valid_labels.shape)

(7509, 224, 224, 3) (7509,)
(260, 224, 224, 3) (260,)
(260, 224, 224, 3) (260,)


In [5]:
# Shuffle the training dataset
train_indices = np.arange(train_images.shape[0])
np.random.shuffle(train_indices)
train_images = train_images[train_indices]
train_labels = train_labels[train_indices]

# Shuffle the validation dataset
valid_indices = np.arange(valid_images.shape[0])
np.random.shuffle(valid_indices)
valid_images = valid_images[valid_indices]
valid_labels = valid_labels[valid_indices]

# Optionally, if you also want to shuffle the test dataset:
test_indices = np.arange(test_images.shape[0])
np.random.shuffle(test_indices)
test_images = test_images[test_indices]
test_labels = test_labels[test_indices]

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Data augmentation
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Callbacks
#checkpoint_cb = ModelCheckpoint("best_model.h5", save_best_only=True)
#early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)
#reduce_lr_cb = ReduceLROnPlateau(factor=0.2, patience=5)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_shape = (224, 224, 3)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 112, 112, 32)      0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 56, 56, 64)       

In [19]:
# Fit the model with a generator
history = model.fit(
    data_gen.flow(train_images, train_labels, batch_size=32),
    epochs=50,  # More epochs
    validation_data=(valid_images, valid_labels)
    
)

Epoch 1/50
235/235 [==============================] - 171s 723ms/step - loss: 1.3994 - accuracy: 0.3554 - val_loss: 1.0612 - val_accuracy: 0.4654
Epoch 2/50
235/235 [==============================] - 176s 749ms/step - loss: 1.1190 - accuracy: 0.4432 - val_loss: 0.9447 - val_accuracy: 0.5308
Epoch 3/50
235/235 [==============================] - 176s 749ms/step - loss: 1.0150 - accuracy: 0.5014 - val_loss: 0.7681 - val_accuracy: 0.6462
Epoch 4/50
235/235 [==============================] - 177s 754ms/step - loss: 0.9041 - accuracy: 0.5691 - val_loss: 0.6646 - val_accuracy: 0.6769
Epoch 5/50
235/235 [==============================] - 162s 688ms/step - loss: 0.8295 - accuracy: 0.5916 - val_loss: 0.5573 - val_accuracy: 0.6962
Epoch 6/50
235/235 [==============================] - 161s 685ms/step - loss: 0.7586 - accuracy: 0.6374 - val_loss: 0.5123 - val_accuracy: 0.7731
Epoch 7/50
235/235 [==============================] - 162s 686ms/step - loss: 0.7435 - accuracy: 0.6448 - val_loss: 0.4964 -

KeyboardInterrupt: 

In [20]:
model.save('PlayingCard.h5')

C:\Users\Fear2\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
model = load_model('PlayingCard.h5')

In [24]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 92ms/step - loss: 0.1106 - accuracy: 0.9577
Test Loss: 0.1106150671839714
Test Accuracy: 0.9576923251152039


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np


predictions = model.predict(valid_images)
predicted_classes = np.argmax(predictions, axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(valid_labels, predicted_classes)

# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=train_class_labels, yticklabels=train_class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

test_loss, test_accuracy = model.evaluate(valid_images, valid_labels)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")



NameError: name 'valid_images' is not defined